In [1]:
from qtaim_embed.models.node_level.base_gnn import GCNNodePred

In [2]:
GCNNodePred(
    atom_input_size=64, 
    bond_input_size=64, 
    global_input_size=64,
    target_dict= {
        "atom": "extra_feat_bond_esp_total"
    },
    conv_fn = "GraphConvDropoutBatch", 
    dropout=0.2, 
    batch_norm=True,
    activation=None,
    bias=True,
    norm="both",
    aggregate="sum",
)

GCNNodePred()

In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy
import networkx as nx

import torch
import dgl
from tqdm import tqdm
from qtaim_embed.utils.grapher import get_grapher
from qtaim_embed.data.molwrapper import mol_wrappers_from_df
from qtaim_embed.utils.tests import get_data
from qtaim_embed.core.dataset import HeteroGraphNodeLabelDataset
from qtaim_embed.data.dataloader import DataLoaderMoleculeNodeTask

df = pd.read_pickle("/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl")


train_dataset = HeteroGraphNodeLabelDataset(
    #file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_full.pkl",
    file="/home/santiagovargas/dev/qtaim_embed/data/xyz_qm8/molecules_qtaim.pkl",
    allowed_ring_size=[3, 4, 5, 6, 7],
    allowed_charges=None,
    self_loop=True,
    extra_keys={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": [
            "extra_feat_bond_esp_total",
            'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta',
            "bond_length",
        ],
        "global": [],
    },
    target_dict={
        "atom": ["extra_feat_atom_esp_total"],
        "bond": ["extra_feat_bond_esp_total", 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta',],
    },
    extra_dataset_info={},
    debug=True,
    log_scale_targets=False,
    standard_scale_targets=True,
)




dataloader = DataLoaderMoleculeNodeTask(train_dataset, batch_size=100, shuffle=True)
batch_graph, batch_label = next(iter(dataloader))

... > running in debug mode
... > creating MoleculeWrapper objects


100%|██████████| 100/100 [00:00<00:00, 4018.26it/s]


element set {'C', 'O', 'N', 'H'}
selected atomic keys ['extra_feat_atom_esp_total']
selected bond keys ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta', 'bond_length']
selected global keys []
... > Building graphs and featurizing


100%|██████████| 100/100 [00:00<00:00, 138.72it/s]


included in labels
{'atom': ['extra_feat_atom_esp_total'], 'bond': ['extra_feat_bond_esp_total', 'extra_feat_bond_ellip_e_dens', 'extra_feat_bond_eta'], 'global': []}
included in graph features
{'atom': ['total_degree', 'total_H', 'is_in_ring', 'ring_size_3', 'ring_size_4', 'ring_size_5', 'ring_size_6', 'ring_size_7', 'chemical_symbol_C', 'chemical_symbol_O', 'chemical_symbol_N', 'chemical_symbol_H'], 'bond': ['metal bond', 'ring inclusion', 'ring size_3', 'ring size_4', 'ring size_5', 'ring size_6', 'ring size_7', 'bond_length'], 'global': ['num atoms', 'num bonds', 'molecule weight']}
... > parsing labels and features in graphs


100%|██████████| 100/100 [00:00<00:00, 3846.43it/s]


... > Scaling features
Standard deviation for feature 0 is 0.0, smaller than 0.001. You may want to exclude this feature.
... > Scaling features complete
... > mean: 
 {'atom': tensor([2.0438, 0.5204, 0.2786, 0.0332, 0.0576, 0.0983, 0.0620, 0.0276, 0.3413,
        0.0802, 0.0657, 0.5128]), 'bond': tensor([0.0000, 0.2825, 0.0349, 0.0637, 0.1011, 0.0662, 0.0300, 1.2698]), 'global': tensor([ 15.9700,  16.3200, 108.9015])}
... > std:  
 {'atom': tensor([1.2534, 0.8735, 0.4483, 0.1791, 0.2330, 0.2977, 0.2411, 0.1637, 0.4741,
        0.2715, 0.2478, 0.4998]), 'bond': tensor([0.0010, 0.4502, 0.1836, 0.2443, 0.3015, 0.2486, 0.1707, 0.2240]), 'global': tensor([2.6700, 2.8562, 6.5972])}
... > Scaling targets
... > Scaling targets complete
... > mean: 
 {'atom': tensor([2084470.5000]), 'bond': tensor([0.9842, 0.0764, 1.4701])}
... > std:  
 {'atom': tensor([5203774.5000]), 'bond': tensor([0.4207, 0.1651, 0.4141])}
... > loaded dataset
